In [13]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import backend as K
from sentence_transformers import SentenceTransformer, util
import torch
from tqdm import tqdm
import pickle


In [ ]:
def save_pickle(model, filename):
    with open(filename, 'wb') as f:
        pickle.dump(model, f)
def load_pickle(filename):
    with open(filename, 'rb') as f:
        return pickle.load(f)

def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))

### The notebook demonstrates how to build  recommender system using retrieval model and ranking model trained before, plus plot embedding in retrieval as well.

## The input is a tuple userId, timestamp and the output is top 5 movie recommendations.

# It solves such question: which movies would you recommend to user X at timestamp Y after user X has watched a list of movies Z? 

#### Load dataset and models, and do preprocessing

In [2]:
data = load_pickle('data_2.pickle')
st_model = SentenceTransformer('all-MiniLM-L6-v2')

# Load retrieval model
neuralCF = tf.keras.models.load_model('neuralCF')
neuralCF.compile(
    loss=rmse,
    optimizer='adam'
)

# Load ranking model 
deepfm = tf.keras.models.load_model('deepfm')
deepfm.compile(
    loss=rmse,
    optimizer='adam'
)

# Load movie dataset
movie = pd.read_csv("u.item", sep='|', header = None, encoding='latin-1', 
                    names = [ "movieId" , "title" , "release_date" , 'video_release_date' ,
              'IMDbURL' , 'unknown' , 'Action' , 'Adventure' , 'Animation' ,
              "Children's" , 'Comedy' , 'Crime' , 'Documentary' , 'Drama' , 'Fantasy',
              'Film-Noir' , 'Horror' , 'Musical' , 'Mystery' , 'Romance' , 'Sci-Fi' ,
              'Thriller' , 'War' , 'Western' ])



2023-03-23 22:06:47.711933: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-23 22:06:47.711973: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2023-03-23 22:06:47.712000: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:163] no NVIDIA GPU device is present: /dev/nvidia0 does not exist
2023-03-23 22:06:47.712227: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
# Apply the same preprocessing rules in model_ranking
data.fillna({ 
             "movie_genre_2":"unknown", 
             'user_fav_genre': "unknown",
             'release_year': data.release_year.mean()}, inplace=True)
# Preprocessing numerical columns
numerical_cols = ['age','IMDb_rating', 'release_year', 'user_avg_rating', 'user_std_rating',  'user_rating_count',
                 'movie_avg_rating', 'movie_std_rating', 'movie_rating_count',]
# Apply normalization techniques to all numerical columns
for column in numerical_cols:
    data[column] = (data[column] - data[column].min()) / (data[column].max() - data[column].min())   

#### Sentence embedding similarity retrieval utility functions

In [4]:
# Store movie embedding in a dedicated df
movie_embedding = data.iloc[data.movieId.drop_duplicates().index][['movieId', 'plot embedding']].reset_index(drop=True)

In [5]:
# Function to return the top 5 movieId that have the most similar plot embedding with input movieId
def get_top5_by_embed(movieId):  
    # Calculate cosine similarity
    cos_sim = util.cos_sim(movie_embedding[movie_embedding.movieId == movieId]['plot embedding'].to_numpy()[0],list(movie_embedding['plot embedding']))
    # Get list of tuple (movieId, similarity) from previous result
    sim_lists = [(idx, sim) for idx, sim in enumerate(cos_sim.numpy()[0])]
    # Sort list by similary descending
    sim_lists.sort(key = lambda x: x[1], reverse = True)
    # Return the top 5 movieId, note I discard the first one since that is the input moiveId
    return [movieId for movieId, sim in sim_lists[1:6]]

In [6]:
# Define features and genre_cols for future uses
features = ['userId', 'age', 'gender', 'occupation', 'zipcode_bucket',                    # User Features
                 'movieId', 'movie_genre_1', 'movie_genre_2', 'IMDb_rating',              # Movie Features
                 'plot embedding', 'release_year',        # Movie Features
                 'rating', 'user_avg_rating', 'user_std_rating', 'user_rating_count',     # Rating Features
                 'movie_avg_rating', 'movie_std_rating', 'movie_rating_count',            # Rating Features
                 'user_fav_genre', 'user_fav_movieId', 'timestamp'                        # Rating Features        
               ]
genre_cols = ['unknown', 'Action', 'Adventure',
       'Animation', "Children's", 'Comedy', 'Crime', 'Documentary', 'Drama',
       'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance',
       'Sci-Fi', 'Thriller', 'War', 'Western']

In [8]:
# Prepare movie timestamp sensitive features
def get_ts_sensitive_features_movie(movieId):
    return data[(data.movieId == movieId) & (data.timestamp <= target_timestamp)].sort_values('timestamp', ascending= False)[timestamp_sensitive_features_movie].iloc[0]

In [9]:
#Constructing a new dataframe for users and picking random timestamps from their watched list

unique_ids = data['userId'].unique()
random_timestamps = []

for id in unique_ids:
    # get all timestamps for a particular user id from the original dataframe
    timestamps = data[data['userId'] == id]['timestamp']
    # select a random timestamp from the list of timestamps for that user id
    random_timestamp = np.random.choice(timestamps)
    random_timestamps.append(random_timestamp)

new_df = pd.DataFrame({'userId': unique_ids,
                       'timestamp': random_timestamps})

In [10]:
new_df

,userId,timestamp
0,1,875071805
1,2,888551853
2,3,889236983
3,4,892002352
4,5,875635372
...,...,...
938,939,880261974
939,940,885921915
940,941,875048996
941,942,891282816


In [16]:

movie_recommendations = {}

for target_userId in tqdm(new_df['userId'].tolist()):
    
    target_timestamp = new_df[new_df['userId'] == target_userId]['timestamp'].iloc[0]
    # These features will change as timestamp changes
    timestamp_sensitive_features_user = ['user_avg_rating', 'user_std_rating', 'user_rating_count','user_fav_genre', 'user_fav_movieId']
    timestamp_sensitive_features_movie = [ 'movie_avg_rating', 'movie_std_rating', 'movie_rating_count']

    # These features will not change as timestamp changes
    timestamp_insensitive_features_user = ['userId', 'age', 'gender', 'occupation', 'zipcode_bucket']
    timestamp_insensitive_features_movie = ['movieId', 'movie_genre_1', 'movie_genre_2', 'IMDb_rating', 'release_year']
    
    # Prepare movie timestamp insensitive features
    movie_retrieval_candidates = data[data.timestamp <=target_timestamp][timestamp_insensitive_features_movie].drop_duplicates()
    
    # Prepare user timestamp insensitive features
    user_ts_insensitive_records = data[data.userId == target_userId][timestamp_insensitive_features_user].drop_duplicates().to_dict('records')[0]

    for f in timestamp_insensitive_features_user:
        movie_retrieval_candidates[f] = user_ts_insensitive_records[f]
        
    # Prepare user timestamp sensitive features
    user_ts_sensitive_records = data[(data.userId == target_userId) & (data.timestamp == target_timestamp)][timestamp_sensitive_features_user].iloc[0].to_dict()

    for f in timestamp_sensitive_features_user:
        movie_retrieval_candidates[f] = user_ts_sensitive_records[f]
    
    movie_retrieval_candidates[timestamp_sensitive_features_movie] = movie_retrieval_candidates.movieId.apply(get_ts_sensitive_features_movie)
    # Get user history
    user_history = data[(data.userId == target_userId) & (data.timestamp <= target_timestamp)]
    
    # Filter movies that this user has already watched from movie candidates
    movie_retrieval_candidates = movie_retrieval_candidates[~movie_retrieval_candidates.movieId.isin(user_history.movieId)]
    
    #NeuralCF
    # Pass movie candidates for retrieval into tf dataset
    retrieval_ds = tf.data.Dataset.from_tensor_slices((dict(movie_retrieval_candidates))).batch(32)
    
    # Predict the rating
    retrieval_result = np.concatenate(neuralCF.predict(retrieval_ds))
    
    # Get indexes of top 50 candidates 
    top50_indexes = np.argpartition(retrieval_result, -50)[-50:]
    
    
    # using plot qmbedding cosine similarity
     
  
    user_recent_5_movies = user_history.sort_values('timestamp', ascending = False).movieId[:5]

    # From these 5 movies, for each movie, get top 5 most similar movies
    # That is 25 movieIds in total
    most_similar_movies_retrieval = set()
    for movieId in user_recent_5_movies:
        most_similar_movies_retrieval.update(get_top5_by_embed(movieId))
    
    # Prepare ranking candidates from retrieval candidates using the top50_indexes result from neuralCF
    movie_ranking_candidates = movie_retrieval_candidates.iloc[top50_indexes]
    
    # Add retrieval result from plot embedding approach
    # First find entries in movie_retrieval_candidates where movieIds are in most_similar_movies_retrieval
    # Then make sure these movieIds are not in movie_ranking_candidates
    movie_retrieval_via_embedding = movie_retrieval_candidates[(movie_retrieval_candidates.movieId.isin(most_similar_movies_retrieval)) 
                                    & 
                                    (~(movie_retrieval_candidates.movieId.isin(movie_ranking_candidates.movieId)))]
    
    # Concatenate two results from retrieval 1 and retrieval 2
    movie_ranking_candidates = pd.concat([movie_retrieval_via_embedding, movie_ranking_candidates])
    
    # Pass movie candidates for ranking into tf dataset
    ranking_ds = tf.data.Dataset.from_tensor_slices((dict(movie_ranking_candidates))).batch(32)
    
    # Predict the rating
    ranking_result = np.concatenate(deepfm.predict(ranking_ds))
        
    # Get top 5 indexes
    top5_indexes = np.argpartition(ranking_result, -5)[-5:]
    
    # Get top 5 movieIds
    top5_movieId = movie_ranking_candidates.iloc[top5_indexes].movieId
    
    movie_recommendations[str(target_userId)] = [movieTitle for movieTitle in movie[movie.movieId.isin(top5_movieId)]['title'].values]  
    

  0%|          | 0/943 [00:00<?, ?it/s]

2/2 [==============================] - 0s 5ms/step


  0%|          | 1/943 [00:01<25:04,  1.60s/it]

3/3 [==============================] - 0s 3ms/step


  0%|          | 2/943 [00:04<34:02,  2.17s/it]

...

100%|█████████▉| 941/943 [36:07<00:04,  2.18s/it]

3/3 [==============================] - 0s 3ms/step


100%|█████████▉| 942/943 [36:10<00:02,  2.29s/it]

3/3 [==============================] - 0s 3ms/step


100%|██████████| 943/943 [36:12<00:00,  2.30s/it]


In [33]:
MR = pd.DataFrame(movie_recommendations)

In [39]:
MR.to_csv("top_5_rec.csv", index=False)

In [50]:

def get_movies_watched_after_timestamp(df, userid, timestamp):
    
    #This function take userid and timestamp and returns all the movies he watched after this time stamp
    
    user_movies = df[df['userId'] == userid] # filter by userid
    user_movies_after_timestamp = user_movies[user_movies['timestamp'] >= timestamp] # filter by timestamp
    return list(user_movies_after_timestamp['title']) # return list of movies

In [79]:
###sanity check

movies_watched = get_movies_watched_after_timestamp(data, 1, new_df[new_df['userId'] == 1]['timestamp'].iloc[0])
movie_rec = movie_recommendations[str(1)]
matches = set(movie_rec).intersection(set(movies_watched))
matches


{'Sling Blade (1996)'}

In [77]:
def calculate_hit_rate(movie_recommendations, sampeling_df, movie_df):
    hits = 0
    for key, value in movie_recommendations.items():
        target_timestamp = sampeling_df[sampeling_df['userId'] == int(key)]['timestamp'].iloc[0]
        movies_user_watched_later = get_movies_watched_after_timestamp(movie_df, int(key), target_timestamp)
        matches = set(value).intersection(set(movies_user_watched_later))
        if len(matches) != 0:
            hits+=1
    return hits/len(movie_recommendations)
        

In [78]:
calculate_hit_rate(movie_recommendations, new_df, data)

0.24602332979851538